TASK 5 and 6: Conversion of the forearm model of last lab to an MDP and Implement a gym environment that behaves like the forearm model, and    which issues quadratic rewards exactly like in the LQR setting of last time

In [12]:
import gymnasium as gym
import numpy as np

class ForeArmModel(gym.Env):
    def __init__ (self) :
        # Define action and observation space
        self.action_space = gym.spaces.Box(low=-12_000, high=2_000, shape=(1,1), dtype=np.float32)

        # Observation space: Angular position, angular velocity
        self.observation_space = gym.spaces.Box(low=-np.inf, high=np.inf, shape=(2,), dtype=np.int64)

        #POMDP somethimes we cannot observe the whole state, its equal to space of x, theta thetadot, a box of shape2 or 2,1 and with bounds whatever, infinity for the bounds
        # theta bet -1 1
        # speed read the max speed to read [-1200,100]
        #reward = quadratic cost
        #state transistion: xk +1 = Axk + bUk
        # example where the agent gets to observe an image 128x128 RGB image
    
    def step (self, action): 
        self.state.reshape([2,1])

        if(self.state.shape == (1,2)):
            self.state = self.state.T

        action = action.reshape([1])

        print(self.state)
        print(self._Q)
        print(self.state.T)
        print(self.state.T @ self._Q)

        temp2 = (self.state.T @ self._Q @ self.state).astype(float)  # Convert to float
        temp3 = (action.T * self._R * action).astype(float)  # Convert to float
        self.reward = (temp2 + temp3)

        temp0 = (self._A @ self.state).reshape([2,1]).T
        temp1 = (self._B * action).T
        self.state = temp0 + temp1

        self.stepCount += 1
        
        obs = self._get_obs()

        if(self.stepCount >= self.MAX_STEP):
            self.is_done = True

        rew = self.reward
        rew = (rew[0])[0]

        return obs, rew, self.is_done, False, { }
        
        
    
    def reset (self, seed=None, options=None):
        super().reset(seed=seed)
        
        self.is_done = False
        self.stepCount = 0
        self.MAX_STEP = 100
        
        #initial condition, could be random, use seed
        x0 = np.array([400, 0])

        self._I = 0.25
        self._b = 0.2
        self.steps = 1
        self._T = 0.01
        self._rho = 1e-3
        self._A = np.array([[1, self._T], [0, 1-self._T*self._b/self._I]]).reshape([2,2])
        self._B = np.array([[0], [self._T/self._I]]).reshape([2,1])
        self._C = np.array([1, 0]).reshape([1, 2])
        self._Q = self._C.T @ self._C
        self._Qf = self._C.T @ self._C
        self._R = self._rho
        self.state = x0.reshape([2,1])

        observation = self._get_obs()
        info = self._get_info()  

        return observation, info
    
    def _get_obs(self):
        tmp = self.state.reshape([2,1])
        ret = np.array([tmp[0][0], tmp[1][0]])
        return ret.astype(np.int64)
        
    def _get_info(self):
        tmp = self.state
        return {"theta": tmp[0][0], "thetap": tmp[1][0]}

    def render(self, mode = 'human', close=False):
        pass

TASK 8

In [13]:
import gymnasium as gym
import numpy as np
from stable_baselines3.common.env_checker import check_env
import matplotlib.pyplot as plt
from stable_baselines3 import PPO

def check_with_lqr (env, lqr):
    state = env.reset()
    x , y , u , costx , costu, J = lqr.simulate()

    assert x[1,0] == env.state[1] #check initial state

    for nx , _x in enumerate(x[1, 1:]):
        action = u[nx].reshape(-1)
        state , reward , is_done , _, _ = env.step(action)
        
        #convert _x into int64
        _x = _x.astype(np.int64)
        
        assert _x == state[1] # check state transitions
        assert reward == J[nx] # check reward

        if is_done:
            break

def check_environment():
    env = ForeArmModel()
    check_env(env)
    
    A, B, C, Q, Qf, R, x0, steps, T = exampleDataLQR()
    lqr = Analytical_LQR(A, B, C, Q, Qf, R, x0, steps, T)
    check_with_lqr(env, lqr)# save the trained agent for reuse

def train_save_model():
    env = ForeArmModel()
    model = PPO("MlpPolicy", env, verbose=1, tensorboard_log="./PPO_ForeArm/") #initiliaze the agent, log and link to tensorboard
    model.learn(total_timesteps=1e5)
    model.save("ppo_forearm")

def load_model_and_check():
    env = ForeArmModel()
    obs = env.reset()
    model = PPO.load("ppo_forearm")
    A, B, C, Q, Qf, R, x0, steps, T = exampleDataLQR()
    lqr = Analytical_LQR(A, B, C, Q, Qf, R, x0, steps, T)
    x , y , u , costx , costu, J = lqr.simulate()


    for nx , _x in enumerate(x[1, 1:]):
        action = u[nx].reshape(-1)
        state, reward, is_done, _, _ = model.predict(obs, deterministic=True)   
        
        #convert _x into int64
        _x = _x.astype(np.int64)
        
        #assert _x == state[1] # check state transitions
        #assert reward == J[nx] # check reward

        if is_done:
            break
    


def main():
    load_model_and_check()


if __name__ == "__main__":
    main()

FileNotFoundError: [Errno 2] No such file or directory: 'ppo_forearm.zip'